In [142]:
import datetime
import psycopg2 as pg
import json
import pandas as pd
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt

In [447]:
def foundFit(plant_num, name_file):
    import cv2 as cv
    import numpy as np
    
    #img0 = cv.imread('{}.png'.format((first_date + days(day_number))))
    #img = cv.imread('{}.png'.format((first_date + days(day_number))))

    img0 = cv.imread(name_file) # reemplazar por la foto de la fecha actual
    img = cv.imread(name_file) # reemplazar por la foto de la fecha actual

    [xLen, yLen] = img.shape[:2]

    ideal_area = xLen * yLen / plant_num

    xL = np.arange(xLen)
    yL = np.arange(yLen)

    R_factor = 1.07
    B_factor = 1.05

    # CREANDO UNA MÁSCARA QUE PINTA DE BLANCO TODO LO VERDE Y NEGRO LO DEMÁS
    for i in xL:
        for j in yL:
            green_px = img0[i, j, 1]
            if green_px > R_factor * img[i, j, 0] and green_px > B_factor * img[i, j, 2]:
                img[i, j, :] = [255, 255, 255]
            else:
                img[i, j, :] = [0, 0, 0]
    
    gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    ret,th1 = cv.threshold(gray_img,200,255,cv.THRESH_BINARY)
    img2,contours,hierarchy = cv.findContours(th1, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    #img3 = cv.drawContours(image, contours, -1, (0,255,0), 3)
    
    fit1 = []
    fit2 = []
    fit3 = []
    
    for c in contours:
        # finally, get the min enclosing circle
        (rx, ry), radius = cv.minEnclosingCircle(c)
        center = (int(rx), int(ry))
        radius = int(radius)
        
        area = cv.contourArea(c)
    
        # and draw the circle in blue
        if radius > 50:
            fit1.append(area / ideal_area)
        
            # get the bounding rect
            y, x, h, w = cv.boundingRect(c)
            # draw a green rectangle to visualize the bounding rect
            #cv.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)  
                        
            mask = np.zeros(img0.shape[:2], np.uint8)
        
            mask[x:x+w, y:y+h] = th1[x:x+w, y:y+h]   
        
            hist_mask_R = cv.calcHist([img0],[2],mask,[256],[0,256])
            hist_mask_G = cv.calcHist([img0],[1],mask,[256],[0,256])
            hist_mask_B = cv.calcHist([img0],[0],mask,[256],[0,256])

            # Number of total frequencies of the actual mask (no matter the channel :p !!!)
            total_mask_fc = np.sum(hist_mask_R)
        
            r_fc = []
            g_fc = []
            b_fc = []
        
            r_val = []
            g_val = []
            b_val = []
        
            for k in np.arange(256):
                r_percentage = hist_mask_R[k][0] / total_mask_fc
                g_percentage = hist_mask_G[k][0] / total_mask_fc
                b_percentage = hist_mask_B[k][0] / total_mask_fc
            
                if r_percentage >= 0.005:
                    r_fc.append(hist_mask_R[k][0])
                    r_val.append(k)
                    
                if g_percentage >= 0.005:
                    g_fc.append(hist_mask_G[k][0])
                    g_val.append(k)
                    
                if b_percentage >= 0.005:
                    b_fc.append(hist_mask_B[k][0])
                    b_val.append(k)
             
            if r_fc == []:
                r_fc = [0]
                r_val = [0]
            
            if g_fc == []:
                g_fc = [0]
                g_val = [0]
                
            if b_fc == []:
                b_fc = [0]
                b_val = [0]
        
            from functools import reduce
            
            # Histogram Frecuencies average
            r_fc_av = reduce((lambda x, y: x + y),r_fc) / 256
            g_fc_av = reduce((lambda x, y: x + y),g_fc) / 256
            b_fc_av = reduce((lambda x, y: x + y),b_fc) / 256
    
            # Histogram Values average
            r_val_av = reduce((lambda x, y: x + y),r_val) / 256
            g_val_av = reduce((lambda x, y: x + y),g_val) / 256
            b_val_av = reduce((lambda x, y: x + y),b_val) / 256
            
            fit2.append(g_val_av / (r_val_av + g_val_av + b_val_av))
            
            fit3.append(g_fc_av / (r_fc_av + g_fc_av + b_fc_av))
            
            # get the min area rect
            rect = cv.minAreaRect(c)
            box = cv.boxPoints(rect)
            # convert all coordinates floating point values to int
            box = np.int0(box)
            # draw a red 'nghien' rectangle
            #cv.drawContours(img, [box], 0, (0, 0, 255))
            
            cv.drawContours(img, [box], 0, (0, 0, 255))
            cv.rectangle(img, (y, x), (y+h, x+w), (0, 255, 0), 2)
            img_final = cv.circle(img, center, radius, (255, 0, 0), 2)
    
    fit1 = reduce((lambda x, y: x + y),fit1) / plant_num
    fit2 = reduce((lambda x, y: x + y),fit2) / plant_num
    fit3 = reduce((lambda x, y: x + y),fit3) / plant_num
    
    fit = [fit1, fit2, fit3]
    return fit

In [143]:
# JALAR TODAS LOS TAGS
con = pg.connect("dbname='raphi_database' user='raphi_role' password='raphi_password' host='localhost'")
cur = con.cursor()
cur.execute("select * from metrics")
rows = cur.fetchall()

In [ ]:
# INGRESAR EL ORDINAL DEL DÍA A EVALUAR
day_number = 1

# Adiciona "d" días a una fecha
def days(d):
    return pd.Timedelta('{} days'.format(d - 1))

first_date = rows[0][3]

df = pd.DataFrame(rows, columns = ["id", "type", "value", "date", "update", "station"])
df = df.set_index(['id'])
df = df[(df['date'] >= (first_date + days(day_number))) & (df['date'] < (first_date + days(day_number+1)))]

# Todos los valores de la base de datos del día evaluado en un objeto lista
dates_mask = df.values.tolist()

In [469]:
env_min = [0, 0, 0, 0, 0, 0, 4, 4, 4, 4] # min_environments_params
env_max = [20, 20, 20, 20, 20, 20, 12, 12, 12, 12] # max_environments_params

In [472]:
def range_percentage(_min, _max, value):
    return (value - _min) / (_max - _min)

In [470]:
# environment params =
# [Airtempday, Airtempnight, Levelday, Levelnight, Lightday, Lightnight, Fshair, FshNutSol, AirCirc, NutSolCirc]
env_parms = [] 

AirTemp = list(filter(lambda x: x[0] == 'AirTemperature', dates_mask))

TnkLevel = list(filter(lambda x: x[0] == 'TankLevel', dates_mask))

Light = list(filter(lambda x: x[0] == 'LightIntensity', dates_mask))

FshAir = list(filter(lambda x: x[0] == 'FreshAir', dates_mask))
FshAir = list(filter(lambda x: x[1] == 'true', FshAir))

FshNutSol = list(filter(lambda x: x[0] == 'FreshNutriSol', dates_mask))
FshNutSol = list(filter(lambda x: x[1] == 'true', FshNutSol))

AirCir = list(filter(lambda x: x[0] == 'AirCirculation', rows))
AirCir = list(filter(lambda x: x[1] == 'true', AirCir))

NutSolCir = list(filter(lambda x: x[0] == 'NutriSolCirculation', dates_mask))
NutSolCir = list(filter(lambda x: x[1] == 'true', NutSolCir))

env_parms.append(AirTemp[0][1])
env_parms.append(AirTemp[len(AirTemp)-1][1])

env_parms.append(TnkLevel[0][1])
env_parms.append(TnkLevel[len(TnkLevel)-1][1])

env_parms.append(Light[0][1])
env_parms.append(Light[len(Light)-1][1])

env_parms.append(len(FshAir))
env_parms.append(len(FshNutSol))
env_parms.append(len(AirCir))
env_parms.append(len(NutSolCir))

env_parms = list(map(float, env_parms))

env_parms_str = str(env_parms)

In [474]:
per_params = list(map(range_percentage, env_min, env_max, env_parms)) # Futuro genoma

In [475]:
# Fecha actual
today = datetime.datetime.now()

#CAPTURA DE FOTO DIARIA
#--- INICIO CAPTURA ---
cap = cv.VideoCapture(0)
readed, frame = cap.read()
if readed == True:
    cv.imwrite("{}.png".format(per_params), frame) # fecha de la foto en el nombre
    #print("Foto tomada correctamente")
else:
    print("Error al acceder a la cámara")
cap.release()
#--- FIN CAPTURA ---

In [ ]:
def make_station(env_parms, Fit):
    fitness_id = str(env_parms)
    return {fitness_id: Fit}

for i in np.arange():
    stations[i] = make_station(env_parms[i], Fit[i])

In [ ]:
import numpy as np

TARGET=[1,1,0];

MUT_PROB = 10;
    
def generateGenome():
    env_min = [0, 0, 0, 0, 0, 0, 4, 4, 4, 4] # min_environments_params
    env_max = [20, 20, 20, 20, 20, 20, 12, 12, 12, 12] # max_environments_params
    
    def generateOneParms(_min, _max):
        return (_min + (_max - _min) * np.random.rand())
    
    def parms_percentage(env_min, env_max, env_per):
        return (env_per - env_min) / (env_max - env_min)
    
    env_parms = list(map(generateOneParms, env_min, env_max)) # O ingresar los valores MANUALMENTE!!!
    
    genome = list(map(parms_percentage, env_min, env_max, env_parms))

    return genome

genome = generateGenome()

# Cálculo del los fitness factors de la imagen diaria respecto del genoma dado
def fitImgFactors(genome):
    #fitness = foundFit(6, '{}.png'.format(genome))
    fitness = foundFit(6, 'letucce.jpeg')
    return fitness

# Obtención del fitness total de la imagen
def getFitness(genome):
    fitness = 0
    fitFactors = fitImgFactors(genome)
    for i in np.arange(len(TARGET)):
        if (TARGET[i]-fitFactors[i] < 0.2):
            fitness = fitness + 1
    return fitness

# Generación de un conjunto ("piscina") de 50 genomas (estaciones) e inicializado con el genoma actual (Estación actual)
def getGenePool(genome):
    pool = []
    for i in np.arange(50):
        pool.append(genome)
    return pool

pool = getGenePool(genome)

# Evaluación de una piscina para obtener el genoma con mayor fitness
def getFittest(pool):
    fittestLoc = 0
    fittest = 0
    for i in np.arange(len(pool)):
        if (getFitness(pool[i]) > fittest):
            fittest = getFitness(pool[i])
            fittestLoc = i
    return pool[fittestLoc]

# Mutación de un genoma (valores nuevos para las condiciones ambientales)
def doMutation(genome):
    testGenome = genome
    newGenome = []
    for i in np.arange(len(genome)):
        if(np.round((np.random.rand()*MUT_PROB))==1):
            testGenome[i]=np.random.rand() # Estos valores deben ser implementados como Setpoint para alguna estación
                                           # esperar a tomar la foto, hallar el fitness 
            if(getFitness(testGenome)>getFitness(genome)):
                newGenome.append(testGenome[i])
            else:
                newGenome.append(genome[i])
        else:
            newGenome.append(genome[i])
    return newGenome

def evolve():
    numGens = 0
    fittest = generateGenome()
    print('AAAAAAA')
    print(fittest)
    while(getFitness(fittest)!=3):
        numGens = numGens + 1
        pool = getGenePool(fittest) # Inicializando nada más
        pool2 = []
        for i in np.arange(len(pool)): # A todos los elementos de pool se les hace mutar para hallar genomas con más fit
            pool2.append(doMutation(pool[i])) # se halla un nuevo pool con genomas con alto fit
        print(pool2)
        fittest = getFittest(pool2) # se selecciona el genoma con mayor fit del pool mejorado(el mejor de lo mejor)
        #if(numGens%10==0):
            #print('BBBBBBBB')
            #print(fittest)
    return fittestStation['genome']

e = evolve();

AAAAAAA
[0.8762978097939417, 0.46394778986173824, 0.7119785099346969, 0.5022233206477273, 0.9027678573450476, 0.8847205327818151, 0.5525412348761931, 0.14503934648674344, 0.06379897776917687, 0.6474942025096451]


In [110]:
datetime_object = datetime.datetime.strptime('2018-12-01', '%Y-%m-%d')

In [111]:
datetime_object + days(2)

datetime.datetime(2018, 12, 2, 0, 0)

In [378]:
myStation

{'[0.0, 21.641475515812004, 12.1, 12.1, 0.0, 1.0, 213.0, 0.0, 0.0, 0.0]': [0.43802401962166276,
  0.7098618594446625,
  0.343119226395153],
 'genome': [0.3699848704503135,
  0.615099752703765,
  0.9979465824259994,
  0.4975496562779371,
  0.21374440291750774,
  0.5912831851571156,
  0.28586868526174025,
  0.09226180908912207,
  0.6642562069270612,
  0.20465433703318425]}

In [373]:
fitImgFactors(myStation)

[0.43802401962166276, 0.7098618594446625, 0.343119226395153]